# Calcul des circonscriptions avec un faible écart aux élections législatives de 2022

In [1]:
import pandas as pd

## premier tour

In [2]:
resultats_2022_t1 = (
    pd
    .read_csv('lg2022_t1_resultats.csv')
)

resultats_2022_t1

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc,CodCirc1,CodCirc2
0,1,GUILLERMIN,Vincent,M.,ENS,Ensemble ! (Majorité présidentielle),8071,"19,30","9,36",non,00101,00101,01001
1,2,LAHY,Éric,M.,DXG,Divers extrême gauche,391,"0,94","0,45",non,00101,00101,01001
2,3,BRETON,Xavier,M.,LR,Les Républicains,10599,"25,35","12,30",Bal.,00101,00101,01001
3,4,PIROUX GIANNOTTI,Brigitte,Mme,RN,Rassemblement National,8971,"21,46","10,41",non,00101,00101,01001
4,5,MENDES,Michael,M.,DSV,Droite souverainiste,641,"1,53","0,74",non,00101,00101,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6285,4,TAPAYEVA,Tamila,Mme,DSV,Droite souverainiste,481,"1,74","0,49",non,09911,09911,ZZ011
6286,5,MARTIN,Catya,Mme,LR,Les Républicains,1760,"6,37","1,78",non,09911,09911,ZZ011
6287,6,VIDAL,Dominique,Mme,NUP,Nouvelle union populaire écologique et sociale,6849,"24,79","6,93",Bal.,09911,09911,ZZ011
6288,7,BURLOTTE,Olivier,M.,RN,Rassemblement National,742,"2,69","0,75",non,09911,09911,ZZ011


In [3]:
def ecarts(x):
    l = x.sort_values(ascending=False).to_list()
    return l[0] - l[1] if len(l) > 1 else None

In [4]:
ecarts_2022_t1 = (
    resultats_2022_t1
    .assign(NbVoix = lambda df: df.NbVoix.astype(int) )
    .groupby('CodCirc2')
    .agg({'NbVoix': ecarts})
)

ecarts_2022_t1.sort_values('NbVoix').head(50)

,NbVoix
CodCirc2,
59005,2
46002,13
81003,27
34003,41
71005,44
80002,51
77006,58
86003,60
94006,65


## second tour

In [5]:
resultats_2022_t2 = pd.read_csv('lg2022_t2_resultats.csv')

resultats_2022_t2

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc,CodCirc1,CodCirc2
0,3,BRETON,Xavier,M.,LR,Les Républicains,24408,"63,22","28,32",oui,00101,00101,01001
1,8,GUÉRAUD,Sébastien,M.,NUP,Nouvelle union populaire écologique et sociale,14203,"36,78","16,48",non,00101,00101,01001
2,1,LAPRAY,Lumir,Mme,NUP,Nouvelle union populaire écologique et sociale,17824,"41,66","17,83",non,00102,00102,01002
3,2,DAUBIÉ,Romain,M.,ENS,Ensemble ! (Majorité présidentielle),24960,"58,34","24,97",oui,00102,00102,01002
4,3,GIVERNET,Olga,Mme,ENS,Ensemble ! (Majorité présidentielle),18398,"58,72","22,38",oui,00103,00103,01003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,3,BEN CHEÏKH,Karim,M.,NUP,Nouvelle union populaire écologique et sociale,11348,"54,07","9,41",oui,09909,09909,ZZ009
1144,4,LAKRAFI,Amélia,Mme,ENS,Ensemble ! (Majorité présidentielle),13048,"63,58","12,49",oui,09910,09910,ZZ010
1145,9,MOUSSA,Chantal,Mme,NUP,Nouvelle union populaire écologique et sociale,7474,"36,42","7,16",non,09910,09910,ZZ010
1146,1,GENETET,Anne,Mme,ENS,Ensemble ! (Majorité présidentielle),16537,"61,73","16,73",oui,09911,09911,ZZ011


In [6]:
gagnants_2022_t2 = (
    resultats_2022_t2
    .pivot_table(
        index = 'CodCirc2',
        columns = 'CodNua',
        values = 'NbVoix'
    )
    .idxmax(axis=1)
    .to_frame(name="En tête")
)

(
    gagnants_2022_t2
    .value_counts()
)

En tête
ENS        244
NUP        127
RN          89
LR          61
DVG         22
DVD         10
REG         10
DVC          4
UDI          3
DIV          1
DSV          1
Name: count, dtype: int64

In [7]:
inscrits_2022 = (
    pd
    .read_csv('lg2022_inscrits.csv')
    .set_index('CodCirc2')
)

inscrits_2022

,inscrits
CodCirc2,
01001,86187
01002,99953
01003,82204
01004,94897
01005,77456
...,...
ZZ007,123409
ZZ008,131219
ZZ009,120556


In [8]:
votants_2022_t2 = (
    resultats_2022_t2
    .groupby('CodCirc2')
    [['NbVoix']]
    .sum()
    .rename({
        'NbVoix': 'votants'
    }, axis=1)
)

votants_2022_t2

,votants
CodCirc2,
01001,38611
01002,42784
01003,31330
01004,36297
01005,30569
...,...
ZZ007,38519
ZZ008,16747
ZZ009,20987


In [9]:
ecarts_2022_t2 = (
    resultats_2022_t2
    .assign(NbVoix = lambda df: df.NbVoix.astype(int) )
    .groupby('CodCirc2')
    .agg({'NbVoix': ecarts})
    .rename({ 'NbVoix': 'écart' }, axis=1)
    .join(votants_2022_t2)
)

On considère que l'écart est faible quand il est inférieur à `5 %` des votants.

In [12]:
ecarts_2022_t2_faible = (
    ecarts_2022_t2
    .query('écart < (votants * 0.05)')
    .join(inscrits_2022)
)

ecarts_2022_t2_faible

,écart,votants,inscrits
CodCirc2,,,
03002,140.0,32092,80970
04001,689.0,28893,61962
04002,926.0,31200,66661
05001,198.0,27336,59643
06002,1188.0,35926,89179
...,...,...,...
ZD007,888.0,35962,118245
ZP001,672.0,38374,72176
ZS001,19.0,2639,5053


On vérifie qu'il n'y a pas de circonscriptions où l'écart est supérieur au nombre d'abstention (`inscrits - votants`).

In [10]:
ecarts_2022_t2_faible.query('écart > (inscrits - votants)')

,écart,votants,inscrits
CodCirc2,,,


In [11]:
ecarts_2022_t2_faible.to_csv('lg2022_t2_ecartsfaibles.csv')